# Sequential global argmin with single particle event

Simulate a set of poisson distributed jets with random distributed $\zeta$ parameter distributed across events, and find out the argmin per event. 

Based on product. Can be trivially extended to pairs.

In [1]:
import numpy

In [4]:
# Data generation step

NUMEVENTS = 100             # Number of evenets to simulate the data for.
AVENUMJETS = 10             # Average number of jets per event.

numjets = numpy.random.poisson(AVENUMJETS, NUMEVENTS).astype(numpy.int) # Number of jets in each event
jets_stops = numpy.cumsum(numjets)                                      # Stops array
jets_starts = numpy.zeros_like(jets_stops)                              # Starts array
jets_starts[1:] = jets_stops[:-1]

jets_counts = jets_stops-jets_starts                            # Count of jets every event 
pairs_offsets = numpy.zeros(NUMEVENTS+1).astype(numpy.int)      # Offsets of product every event
pairs_offsets[1:] = numpy.cumsum(jets_counts*jets_counts)

# Define zeta values. Drawn from a uniform distribution.
jets_zeta = numpy.random.uniform(low=0, high=10, size=(pairs_offsets[-1],1))

### Argmin calculation step

This can be done in steps as below:

1. Iterate over all events, indexed by i.
2. For each event, 
    - create a dummy variable for storing the minimum value, and initialize it.
    - iterate over the jets 1(index j) and jets 2(index k).
    - If current jets_zeta <= global jets_zets ( from minimum value), update minimum index variables `left[i]=j` and `right[i]=k`.
    - Carry this for all acceptable values of j and k.
3. Store the minimum index for each event in `left` and `right` arrays.    

In [9]:
# Now find global argmin sequentially.
# First let's create two arrays for storing the min indexes for each event. 
# left for jet1, right for jets2.
# Minimum zeta is stored in min_zeta variable.

left = numpy.empty(NUMEVENTS, dtype=numpy.int32)
right = numpy.empty(NUMEVENTS, dtype=numpy.int32)
min_zeta = numpy.empty(NUMEVENTS, dtype=numpy.int32)

for i in range(NUMEVENTS):
    pairs_i = pairs_offsets[i]
    min_val = jets_zeta[pairs_i]
    for j in range(jets_starts[i], jets_stops[i]):
        for k in range(jets_starts[i], jets_stops[i]):
            if jets_zeta[pairs_i] <= min_val :
                min_val = jets_zeta[pairs_i]
                left[i] = j
                right[i] = k
                min_zeta[i] = min_val
            pairs_i +=1

In [10]:
# Print the values for first 6 events

for i in range(6):
    print("Event {}:   Global minumum index: {}  Global Minimum Zeta: {}".format(i, (left[i], right[i]), min_zeta[i]))

Event 0:   Global minumum index: (4, 7)  Global Minimum Zeta: 0
Event 1:   Global minumum index: (13, 17)  Global Minimum Zeta: 0
Event 2:   Global minumum index: (25, 26)  Global Minimum Zeta: 0
Event 3:   Global minumum index: (34, 38)  Global Minimum Zeta: 0
Event 4:   Global minumum index: (44, 41)  Global Minimum Zeta: 0
Event 5:   Global minumum index: (59, 58)  Global Minimum Zeta: 0


Note that minimum zeta is 0 in case of product for single particle. This is because each particle is repeated in the two tracks j and k.